In [8]:
import folium
import numpy as np
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from pyffp import calc_footprint_FFP_climatology as myfootprint

from pyffp.utils import get_dd
from pyffp.utils import contour_to_gdf

In [2]:
# define footprint inputs
zmt = 20.0
z0t = 0.01
ht = [2000.0, 1800.0, 1500.0]
olt = [-10.0, -100.0, -500.0]
sigmavt = [
    0.9,
    0.7,
    0.3,
]
ustart = [0.5, 0.3, 0.4]
wind_dirt = [30.0, 50.0, 70.0]
domaint = [-100.0, 1000.0, -100.0, 1000.0]
nxt = 1100
rst = [20.0, 40.0, 60.0, 80.0]

In [3]:
# generate footprints
FFP = myfootprint.FFP_climatology(
    zm=zmt,
    z0=z0t,
    umean=None,
    h=ht,
    ol=olt,
    sigmav=sigmavt,
    ustar=ustart,
    wind_dir=wind_dirt,
    domain=domaint,
    nx=nxt,
    rs=rst,
    smooth_data=1
)


Alert(0017):
 Only one value of zm passed. Using it for all footprints.
 Execution continues.

Alert(0013):
 Using z0, ignoring umean if passed.
 Execution continues.

Calculating footprint  1  of  3
Calculating footprint  2  of  3
Calculating footprint  3  of  3


In [4]:
# georeference grids
x_2d = FFP["x_2d"]
y_2d = FFP["y_2d"]
# set the origin at the SE-Nor station
origin_lon = 17.4795
origin_lat = 60.0865
x_2d_dd = np.vectorize(get_dd)(x_2d) + origin_lon
y_2d_dd = np.vectorize(get_dd)(y_2d) + origin_lat

In [5]:
# prepare contour inputs
clevs = FFP["fr"][::-1]
clevs = [clev for clev in clevs if clev is not None]
levs = [clev for clev in clevs]
fs = FFP["fclim_2d"]
cs = [cm.jet(ix) for ix in np.linspace(0, 1, len(fs))]
f = fs[0]
c = cs[0]
cc = [c]*len(levs)

In [6]:
# generate GeoDataFrame of contours
fig, ax = plt.subplots(figsize=(10, 8))
cp = ax.contour(x_2d_dd, y_2d_dd, fs, levs, colors = cc, linewidths=0.5)
gdf = contour_to_gdf(cp)
plt.close()

In [13]:
# plotting
origin = gpd.GeoDataFrame(
    {"site": "SE-Nor"},
    geometry=gpd.points_from_xy(x=[origin_lon], y=[origin_lat]),
    crs=4326,
    index=[0],
).to_json()
origin = folium.features.GeoJson(origin)

contours = gdf.to_json()
contours = folium.features.GeoJson(contours)

map_osm = folium.Map(location=[origin_lat, origin_lon], zoom_start=16)
map_osm.add_child(origin)
map_osm.add_child(contours)
map_osm